In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# import
import keras
import keras.layers
from keras.models import Sequential
from keras.optimizers import Adam
from matplotlib import pyplot as plt

In [ ]:
# load training and test data
train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv').to_numpy()
test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv').to_numpy()

In [ ]:
X_train = train[:,1:]
Y_train = train[:,0]
X_test = test

In [ ]:
# display training data
train_index = 23
plt.imshow(np.reshape(X_train[train_index], (28, 28)))

In [ ]:
X_train.shape

In [ ]:
# normalize
X_train_normalized = X_train / 255.0
X_test_normalized = X_test / 255.0

In [ ]:
# reshape
X_train_normalized = X_train_normalized.reshape((-1, 28, 28, 1))
X_test_normalized = X_test_normalized.reshape((-1, 28, 28, 1))

In [ ]:
X_train_normalized.shape

In [ ]:
# Define the plotting function
def plot_curve(epochs, hist, list_of_metrics):
  """Plot a curve of one or more classification metrics vs. epoch."""  
  # list_of_metrics should be one of the names shown in:
  # https://www.tensorflow.org/tutorials/structured_data/imbalanced_data#define_the_model_and_metrics  

  plt.figure()
  plt.xlabel("Epoch")
  plt.ylabel("Value")

  for m in list_of_metrics:
    x = hist[m]
    plt.plot(epochs[1:], x[1:], label=m)

  plt.legend()

print("Loaded the plot_curve function.")

In [ ]:
def create_model(learning_rate):
    """Create and compile a deep neural net."""
    # Sequential model
    model = keras.models.Sequential()
    
    # Convolutional layers
    model.add(keras.layers.Conv2D(32, 3, activation='relu', padding='same', input_shape=(28,28,1)))
    model.add(keras.layers.MaxPooling2D((2,2)))
    model.add(keras.layers.Conv2D(32, 3, activation='relu'))
    model.add(keras.layers.Conv2D(32, 3, activation='relu'))
    model.add(keras.layers.Conv2D(32, 3, activation='relu',))
    model.add(keras.layers.MaxPooling2D((2,2)))
    
    # Flatten
    model.add(keras.layers.Flatten())

    # Dense layers followed by dropout   
    model.add(keras.layers.Dense(units=512, activation='relu')) 
    model.add(keras.layers.Dropout(rate=0.5))
    model.add(keras.layers.Dense(units=256, activation='relu')) 
    model.add(keras.layers.Dropout(rate=0.5))
    

    # Output layer with 10 units
    model.add(keras.layers.Dense(units=10, activation='softmax'))     

    # Compile model with categorical cross-entropy loss
    model.compile(optimizer=keras.optimizers.Adam(lr=learning_rate),
                loss="sparse_categorical_crossentropy",
                metrics=['accuracy'])

    return model    


def train_model(model, train_features, train_label, epochs,
                batch_size=None, validation_split=0.1):
  """Train the model by feeding it data."""

  history = model.fit(x=train_features, y=train_label, batch_size=batch_size,
                      epochs=epochs, shuffle=True, 
                      validation_split=validation_split)
 
  # To track the progression of training, gather a snapshot
  # of the model's metrics at each epoch. 
  epochs = history.epoch
  hist = pd.DataFrame(history.history)

  return epochs, hist

In [ ]:
# The following variables are the hyperparameters.
learning_rate = 0.003
epochs = 60
batch_size = 4000
validation_split = 0.2

# Establish the model's topography.
my_model = create_model(learning_rate)

In [ ]:
my_model.summary()

In [ ]:
# Train the model on the normalized training set.
epochs, hist = train_model(my_model, X_train_normalized, Y_train, 
                           epochs, batch_size, validation_split)

# Plot a graph of the metric vs. epochs.
list_of_metrics_to_plot = ['accuracy', 'val_accuracy']
plot_curve(epochs, hist, list_of_metrics_to_plot)

In [ ]:
# Predict on test data
predictions = np.argmax(my_model.predict(X_test_normalized), axis=1)

In [ ]:
# write predictions to csv
example_indices = np.reshape(np.arange(1,predictions.size+1),(-1,1))
headers = np.array([['ImageId', 'Label']])
predictions_table = np.concatenate((headers, np.concatenate((example_indices, np.reshape(predictions, (-1,1))), axis=1)), axis=0)

np.savetxt('predictions.csv', predictions_table, delimiter=',', fmt='%s')